In [3]:
import os
from keras.preprocessing.image import  ImageDataGenerator
from keras.layers import Dense,Flatten,GlobalAveragePooling2D,Input,Conv2D,MaxPooling2D,Dropout
from keras.models import Model,load_model,Sequential
from keras.applications.xception import  Xception
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import  ResNet50
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.mobilenet import MobileNet
import keras.callbacks as kcallbacks
import keras

from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
#from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import image_utils


import math
import random
from keras.utils import plot_model
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
import numpy as np
from PIL import Image
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

2023-07-25 06:07:16.720475: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-25 06:07:18.961937: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-07-25 06:07:18.962292: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-07-25 06:07:18.962305: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [4]:
#generate training and test images
TARGET_SIZE=(224,224)
INPUT_SIZE=(224,224,3)        #输入大小设为（224,224,3）
BATCHSIZE=128	#could try 128 or 32  #批量大小设置为128

#Normalization
#ImageDataGenerator，rescale的作用是对图片的每个像素值均乘上这个放缩因子，这个操作在所有其它变换操作之前执行，在一些模型当中，
#直接输入原图的像素值可能会落入激活函数的“死亡区”，因此设置放缩因子为1/255，把像素值放缩到0和1之间有利于模型的收敛，避免神经元“死亡”。
train_datagen = ImageDataGenerator(rescale=1./255)  ##归一化

test_datagen = ImageDataGenerator(rescale=1./255)   # #不增强验证数据

train_generator = train_datagen.flow_from_directory(  #从指定的目录中生成训练图像
        './dataset/train_224',##训练文件路径--文件夹下多分类文件夹
        target_size=TARGET_SIZE,                     #目标大小设置为(224,224)，图像统一尺寸
        batch_size=BATCHSIZE,                        #batch数据的大小,默认32
        class_mode='categorical')                   #返回标签数组形式 'binary'二进制；
validation_generator = test_datagen.flow_from_directory(  #从指定的目录中生成测试图像
        './dataset/test_224',
        target_size=TARGET_SIZE,
        batch_size=BATCHSIZE,
        class_mode='categorical')

Found 4313 images belonging to 5 classes.
Found 1024 images belonging to 5 classes.


In [5]:
#plot the figures
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = {'batch': [], 'epoch': []}
        self.accuracy = {'batch': [], 'epoch': []}
        self.val_loss = {'batch': [], 'epoch': []}
        self.val_acc = {'batch': [], 'epoch': []}

    def on_batch_end(self, batch, logs={}):
        self.losses['batch'].append(logs.get('loss'))
        self.accuracy['batch'].append(logs.get('accuracy'))
        self.val_loss['batch'].append(logs.get('val_loss'))
        self.val_acc['batch'].append(logs.get('val_accuracy'))

    def on_epoch_end(self, batch, logs={}):
        self.losses['epoch'].append(logs.get('loss'))
        self.accuracy['epoch'].append(logs.get('accuracy'))
        self.val_loss['epoch'].append(logs.get('val_loss'))
        self.val_acc['epoch'].append(logs.get('val_accuracy'))

    def loss_plot(self, loss_type):
        iters = range(len(self.losses[loss_type]))
        #创建一个图
        plt.figure()
        # acc
        plt.plot(iters, self.accuracy[loss_type], 'r', label='train acc')#plt.plot(x,y)，这个将数据画成曲线
        # loss
        plt.plot(iters, self.losses[loss_type], 'g', label='train loss')
        if loss_type == 'epoch':
            # val_acc
            plt.plot(iters, self.val_acc[loss_type], 'b', label='val acc')
            # val_loss
            plt.plot(iters, self.val_loss[loss_type], 'k', label='val loss')
        plt.grid(True)#设置网格形式
        plt.xlabel(loss_type)
        plt.ylabel('acc-loss')#给x，y轴加注释
        plt.legend(loc="upper right")#设置图例显示位置
        plt.savefig('acc-loss.jpg', dpi=800)
        plt.show()

In [6]:
#创建一个LossHistory的实例
history_this= LossHistory()

In [15]:
!pip install --upgrade pip

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [16]:
!pip install --index-url https://pypi.org/simple hyperopt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 1.2 MB/s eta 0:00:00a 0:00:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.9/840.9 kB 12.3 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 12.8 MB/s eta 0:00:00
  Created wheel for future: filename=future-0.18.3-py3-none-any.whl size=492025 sha256=4b4a87f1607dd83c19964856c8519d4012fd33f8d92c1b3c4546fae3dcfa30ff
  Stored in directory: /home/featurize/.cache/pip/wheels/a0/0b/ee/e6994fadb42c1354dcccb139b0bf2795271bddfe6253ccdf11
Successfully built future


In [7]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

def resnet(num_class, epochs=20,frozen=120,patience=2, dropout_rate=0.5,verbose=0,lr=0.000001, savepath='./resnet.h5', history=history_this, input_shape=INPUT_SIZE):
    model_fine_tune = ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)
    for layer in model_fine_tune.layers[:frozen]:
        layer.trainable = False
    for layer in model_fine_tune.layers[frozen:]:
        layer.trainable = True
    model = GlobalAveragePooling2D()(model_fine_tune.output)
    model = Dense(units=256, activation='relu')(model)
    model = Dropout(dropout_rate)(model)
    model = Dense(num_class, activation='softmax')(model)
    model = Model(model_fine_tune.input, model, name='resnet')
    opt = keras.optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    earlyStopping = kcallbacks.EarlyStopping(
        monitor='val_accuracy', patience=patience, verbose=1, mode='auto')
    saveBestModel = kcallbacks.ModelCheckpoint(
        filepath=savepath,
        monitor='val_accuracy',
        verbose=1,
        save_best_only=True,
        mode='auto')
    hist = model.fit_generator(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        use_multiprocessing=True,
        callbacks=[earlyStopping, saveBestModel, history]
    )
    return hist, model

def prediction(resnet_model):
    # read images from validation folder
    rootdir = './dataset/test_224/'
    test_labels = []
    test_images = []
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            if not (file.endswith(".jpeg") or file.endswith(".jpg") or file.endswith(".png")):
                continue
            test_labels.append(subdir.split('/')[-1])
            test_images.append(os.path.join(subdir, file))

    predict = []
    length = len(test_images)
    label = validation_generator.class_indices
    label = {v: k for k, v in label.items()}
    for i in range(length):
        inputimg = test_images[i]
        test_batch = []
        thisimg = np.array(Image.open(inputimg)) / 255
        test_shape = (1,) + thisimg.shape
        thisimg = thisimg.reshape(test_shape)
        resnet_model_batch = resnet_model.predict(thisimg)
        prob = resnet_model_batch[0, np.argmax(resnet_model_batch, axis=1)[0]]
        res = label[np.argmax(resnet_model_batch, axis=1)[0]]
        predict.append(res)
    acc = accuracy_score(test_labels, predict)
    return acc

def objective(params):
    params = {
        'epochs': int(params['epochs']),
        'patience': int(params['patience']),
        'lr': abs(float(params['lr'])),
        'dropout_rate': abs(float(params['dropout_rate'])),
    }
    epochs = params['epochs']
    patience = params['patience']
    lr = params['lr']
    dropout_rate = params['dropout_rate']

    hist, model = resnet(num_class=5, epochs=epochs, patience=patience, lr=lr, dropout_rate=dropout_rate)
    acc = prediction(resnet_model=model)

    print('accuracy: %s' % acc)
    return {'loss': -acc, 'status': STATUS_OK, 'accuracy': acc}



In [8]:
import time
# Hyperparameter optimization by Bayesian optimization - Tree Parzen Estimator
space = {
    'frozen': hp.quniform('frozen', 120, 150, 15),
    'epochs': hp.quniform('epochs', 15, 31, 5),
    'patience': hp.quniform('patience', 2, 4, 1),
    'lr': hp.quniform('lr', 0.000001, 0.000006, 0.000001),
    'dropout_rate': hp.quniform('dropout_rate', 0.3, 0.6, 0.1),
}

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Disable TensorFlow logs

# Rest of your code

t1 = time.time()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10,
            trials=Trials(),
            verbose=0)  # Set verbose=0 to suppress output
t2 = time.time()

print("Hyperopt estimated optimum {}".format(best))
print("Time: " + str(t2 - t1))



/environment/miniconda3/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
/tmp/ipykernel_1041275/958624459.py:24: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = model.fit_generator(


Epoch 1/25
34/34 [==============================] - ETA: 0s - loss: 0.7036 - accuracy: 0.7797
Epoch 1: val_accuracy improved from -inf to 0.16113, saving model to ./resnet.h5
34/34 [==============================] - 28s 550ms/step - loss: 0.7036 - accuracy: 0.7797 - val_loss: 1.8202 - val_accuracy: 0.1611
Epoch 2/25
34/34 [==============================] - ETA: 0s - loss: 0.1830 - accuracy: 0.9622
Epoch 2: val_accuracy did not improve from 0.16113
34/34 [==============================] - 16s 459ms/step - loss: 0.1830 - accuracy: 0.9622 - val_loss: 1.7960 - val_accuracy: 0.0000e+00
Epoch 3/25
34/34 [==============================] - ETA: 0s - loss: 0.1033 - accuracy: 0.9808
Epoch 3: val_accuracy did not improve from 0.16113
34/34 [==============================] - 16s 463ms/step - loss: 0.1033 - accuracy: 0.9808 - val_loss: 1.7415 - val_accuracy: 0.0537
Epoch 4/25
34/34 [==============================] - ETA: 0s - loss: 0.0708 - accuracy: 0.9875
Epoch 4: val_accuracy improved from 0.161

/environment/miniconda3/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
/tmp/ipykernel_1041275/958624459.py:24: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = model.fit_generator(


Epoch 1/20
34/34 [==============================] - ETA: 0s - loss: 0.8428 - accuracy: 0.7389
Epoch 1: val_accuracy improved from -inf to 0.17676, saving model to ./resnet.h5
34/34 [==============================] - 25s 639ms/step - loss: 0.8428 - accuracy: 0.7389 - val_loss: 1.7564 - val_accuracy: 0.1768
Epoch 2/20
34/34 [==============================] - ETA: 0s - loss: 0.2959 - accuracy: 0.9427
Epoch 2: val_accuracy did not improve from 0.17676
34/34 [==============================] - 16s 461ms/step - loss: 0.2959 - accuracy: 0.9427 - val_loss: 1.6791 - val_accuracy: 0.1768
Epoch 3/20
34/34 [==============================] - ETA: 0s - loss: 0.1779 - accuracy: 0.9699
Epoch 3: val_accuracy improved from 0.17676 to 0.17773, saving model to ./resnet.h5
34/34 [==============================] - 21s 612ms/step - loss: 0.1779 - accuracy: 0.9699 - val_loss: 1.6274 - val_accuracy: 0.1777
Epoch 4/20
34/34 [==============================] - ETA: 0s - loss: 0.1233 - accuracy: 0.9801
Epoch 4: val

/environment/miniconda3/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
/tmp/ipykernel_1041275/958624459.py:24: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = model.fit_generator(


...

Epoch 1/20
34/34 [==============================] - ETA: 0s - loss: 0.6840 - accuracy: 0.7948
Epoch 1: val_accuracy improved from -inf to 0.06348, saving model to ./resnet.h5
34/34 [==============================] - 23s 545ms/step - loss: 0.6840 - accuracy: 0.7948 - val_loss: 1.8715 - val_accuracy: 0.0635
Epoch 2/20
34/34 [==============================] - ETA: 0s - loss: 0.1949 - accuracy: 0.9571
Epoch 2: val_accuracy improved from 0.06348 to 0.22559, saving model to ./resnet.h5
34/34 [==============================] - 18s 532ms/step - loss: 0.1949 - accuracy: 0.9571 - val_loss: 1.7925 - val_accuracy: 0.2256
Epoch 3/20
34/34 [==============================] - ETA: 0s - loss: 0.1116 - accuracy: 0.9768
Epoch 3: val_accuracy improved from 0.22559 to 0.27539, saving model to ./resnet.h5
34/34 [==============================] - 18s 525ms/step - loss: 0.1116 - accuracy: 0.9768 - val_loss: 1.7350 - val_accuracy: 0.2754
Epoch 4/20
34/34 [==============================] - ETA: 0s - loss: 0.072

/environment/miniconda3/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
/tmp/ipykernel_1041275/958624459.py:24: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = model.fit_generator(


Epoch 1/25
34/34 [==============================] - ETA: 0s - loss: 0.5921 - accuracy: 0.8226
Epoch 1: val_accuracy improved from -inf to 0.30566, saving model to ./resnet.h5
34/34 [==============================] - 24s 557ms/step - loss: 0.5921 - accuracy: 0.8226 - val_loss: 1.6642 - val_accuracy: 0.3057
Epoch 2/25
34/34 [==============================] - ETA: 0s - loss: 0.1515 - accuracy: 0.9617
Epoch 2: val_accuracy did not improve from 0.30566
34/34 [==============================] - 17s 487ms/step - loss: 0.1515 - accuracy: 0.9617 - val_loss: 1.6203 - val_accuracy: 0.3047
Epoch 3/25
34/34 [==============================] - ETA: 0s - loss: 0.0874 - accuracy: 0.9812
Epoch 3: val_accuracy did not improve from 0.30566
34/34 [==============================] - 21s 612ms/step - loss: 0.0874 - accuracy: 0.9812 - val_loss: 1.5776 - val_accuracy: 0.2939
Epoch 3: early stopping
1/1 [==============================] - 0s 22ms/step
accuracy: 0.2939453125


/environment/miniconda3/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
/tmp/ipykernel_1041275/958624459.py:24: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = model.fit_generator(


Epoch 1/25
34/34 [==============================] - ETA: 0s - loss: 0.6686 - accuracy: 0.7846
Epoch 1: val_accuracy improved from -inf to 0.19531, saving model to ./resnet.h5
34/34 [==============================] - 25s 648ms/step - loss: 0.6686 - accuracy: 0.7846 - val_loss: 1.7214 - val_accuracy: 0.1953
Epoch 2/25
34/34 [==============================] - ETA: 0s - loss: 0.1737 - accuracy: 0.9613
Epoch 2: val_accuracy did not improve from 0.19531
34/34 [==============================] - 16s 460ms/step - loss: 0.1737 - accuracy: 0.9613 - val_loss: 1.6845 - val_accuracy: 0.0771
Epoch 3/25
34/34 [==============================] - ETA: 0s - loss: 0.0904 - accuracy: 0.9847
Epoch 3: val_accuracy did not improve from 0.19531
34/34 [==============================] - 17s 473ms/step - loss: 0.0904 - accuracy: 0.9847 - val_loss: 1.6129 - val_accuracy: 0.1123
Epoch 4/25
34/34 [==============================] - ETA: 0s - loss: 0.0563 - accuracy: 0.9905
Epoch 4: val_accuracy did not improve from 0.